In [1]:
import fastf1 as ff1
import plotly.graph_objects as go
import plotly.express as px
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import matplotlib as mpl

In [2]:
year = 2024
wknd = 'Monaco'
ses = 'R'
colormap = mpl.cm.plasma

In [3]:
session = ff1.get_session(year, wknd, ses)
weekend = session.event
session.load()

req         WARNING 	DEFAULT CACHE ENABLED! (132.86 MB) C:\Users\rafam\AppData\Local\Temp\fastf1
core           INFO 	Loading data for Monaco Grand Prix - Race [v3.4.4]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['16', '81', '55', '4', '63', '1', '44', '22', '23', '10', '14', '3', '77', '18', '

Vamos plotar as 5 voltas mais rápidas!

In [4]:
# sort by laptime
best_5 = session.laps.sort_values(by='LapTime').drop_duplicates(subset="Driver", keep='first').head(5) 
best_5

,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,Sector2Time,...,FreshTyre,Team,LapStartTime,LapStartDate,TrackStatus,Position,Deleted,DeletedReason,FastF1Generated,IsAccurate
530,0 days 02:59:42.350000,HAM,44,0 days 00:01:14.165000,63.0,3.0,NaT,NaT,0 days 00:00:19.548000,0 days 00:00:34.854000,...,False,Mercedes,0 days 02:58:28.185000,2024-05-26 15:06:33.574,1,7.0,False,,False,True
447,0 days 02:53:20.819000,VER,1,0 days 00:01:14.569000,58.0,3.0,NaT,NaT,0 days 00:00:19.343000,0 days 00:00:35.377000,...,False,Red Bull Racing,0 days 02:52:06.250000,2024-05-26 15:00:11.639,1,6.0,False,,False,True
1231,0 days 03:17:59.654000,ZHO,24,0 days 00:01:14.718000,75.0,3.0,NaT,NaT,0 days 00:00:19.751000,0 days 00:00:34.990000,...,True,Kick Sauber,0 days 03:16:44.936000,2024-05-26 15:24:50.325,1,16.0,False,,False,True
622,0 days 03:19:01.054000,TSU,22,0 days 00:01:14.720000,77.0,2.0,NaT,NaT,0 days 00:00:19.503000,0 days 00:00:35.233000,...,False,RB,0 days 03:17:46.334000,2024-05-26 15:25:51.723,1,8.0,False,,False,True
217,0 days 02:58:03.324000,SAI,55,0 days 00:01:14.726000,62.0,2.0,NaT,NaT,0 days 00:00:19.553000,0 days 00:00:35.324000,...,True,Ferrari,0 days 02:56:48.598000,2024-05-26 15:04:53.987,1,3.0,False,,False,True


In [5]:
best_5.iloc[0].telemetry.head()

,Date,SessionTime,DriverAhead,DistanceToDriverAhead,Time,RPM,Speed,nGear,Throttle,Brake,DRS,Source,Distance,RelativeDistance,Status,X,Y,Z
2,2024-05-26 15:06:33.574,0 days 02:58:28.185000,,250.320556,0 days 00:00:00,11047,267,7,100,False,0,interpolation,0.023088,0.000007,OnTrack,-7661,-6607,502
3,2024-05-26 15:06:33.591,0 days 02:58:28.202000,,250.320556,0 days 00:00:00.017000,11071,268,7,100,False,0,pos,1.296757,0.000394,OnTrack,-7663,-6595,502
4,2024-05-26 15:06:33.771,0 days 02:58:28.382000,,250.320556,0 days 00:00:00.197000,11120,269,7,100,False,0,pos,14.822514,0.004502,OnTrack,-7671,-6460,502
5,2024-05-26 15:06:33.891,0 days 02:58:28.502000,,250.320556,0 days 00:00:00.317000,11168,270,7,100,False,0,pos,23.880090,0.007252,OnTrack,-7676,-6370,503
6,2024-05-26 15:06:33.941,0 days 02:58:28.552000,,250.320556,0 days 00:00:00.367000,11217,271,7,100,False,0,car,27.663611,0.008401,OnTrack,-7677,-6332,503


In [6]:
import fastf1.plotting as ff1plt
def plot_laps(laps, session):
    fig = go.Figure()

    for i, lap in laps.iterrows():
        driver_color = ff1plt.get_driver_color(lap["Driver"], session)
        
        # Plot the lap line
        fig.add_trace(go.Scatter(
            x=lap.telemetry['X'],
            y=lap.telemetry['Y'],
            mode='lines',
            line=dict(color=driver_color, width=1),
            name=f"{lap['Driver']} - {lap['LapTime'].total_seconds():.3f}s"
        ))
        
        # Plot off-track points
        off_track = lap.telemetry[lap.telemetry['Status'] == 'OffTrack']
        if not off_track.empty:
            fig.add_trace(go.Scatter(
                x=off_track['X'],
                y=off_track['Y'],
                mode='markers',
                marker=dict(color=driver_color, size=8),
                name=f"{lap['Driver']} Off Track",
                showlegend=False
            ))

    fig.update_layout(
        title="Lap Comparison",
        xaxis_title="X",
        yaxis_title="Y",
        showlegend=True,
        width=1200,
        height=675,
        paper_bgcolor='rgba(0, 0, 0, 0.9)',
        plot_bgcolor='rgba(0, 0, 0, 0.9)',
        font=dict(color='white'),
        xaxis=dict(
            showgrid=True,
            gridcolor='rgba(128, 128, 128, 0.2)',
            zeroline=False,
            color='white'
        ),
        yaxis=dict(
            showgrid=True,
            gridcolor='rgba(128, 128, 128, 0.2)',
            zeroline=False,
            color='white'
        )
    )

    fig.show()

plot_laps(best_5, session)

req            INFO 	Using cached data for driver_info
core        WARNING 	Failed to preserve data type for column 'X' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Y' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Z' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'X' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Y' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Z' while merging telemetry.


Pontos percebidos:
- Número de coordenadas não é constante, podem existir voltas com mais ou menos coordenadas registradas;
- Utilizar gráfico desta maneira é muito feio e até impreciso, já que algumas conexões entre pontos são feitas cortando parte do circuito;
- Essa visualização, apesar de possibilitar ver pontos fora da pista, não é muito útil sem os limites de pista.

Além desses pontos, também temos que pensar que apenas olhar o traçado seria insuficiente. Diferentes carros vão ter várias diferenças entre si, então terei que usar de qualquer jeito as outras informações que a api fornece.

In [7]:
import fastf1.plotting as ff1plt
def plot_laps(laps, session):
    fig = go.Figure()

    for i, lap in laps.iterrows():
        driver_color = ff1plt.get_driver_color(lap["Driver"], session)
        
        # Plot the lap line
        fig.add_trace(go.Scatter(
            x=lap.telemetry['X'],
            y=lap.telemetry['Y'],
            mode='lines+markers',
            line=dict(color=driver_color, width=1),
            marker=dict(
                color=driver_color,
                size=12,  # Much larger points
                opacity=0.8,  # Less transparency to make points more visible
                symbol='circle'
            ),
            name=f"{lap['Driver']} - {lap['LapTime'].total_seconds():.3f}s"
        ))
        
        # Plot off-track points
        off_track = lap.telemetry[lap.telemetry['Status'] == 'OffTrack']
        if not off_track.empty:
            fig.add_trace(go.Scatter(
                x=off_track['X'],
                y=off_track['Y'],
                mode='markers',
                marker=dict(color=driver_color, size=8),
                name=f"{lap['Driver']} Off Track",
                showlegend=False
            ))

    fig.update_layout(
        title="Lap Comparison",
        xaxis_title="X",
        yaxis_title="Y",
        showlegend=True,
        width=1200,
        height=675,
        paper_bgcolor='rgba(0, 0, 0, 0.9)',
        plot_bgcolor='rgba(0, 0, 0, 0.9)',
        font=dict(color='white'),
        xaxis=dict(
            showgrid=True,
            gridcolor='rgba(128, 128, 128, 0.2)',
            zeroline=False,
            color='white'
        ),
        yaxis=dict(
            showgrid=True,
            gridcolor='rgba(128, 128, 128, 0.2)',
            zeroline=False,
            color='white'
        )
    )

    fig.show()

plot_laps(best_5, session)

core        WARNING 	Failed to preserve data type for column 'X' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Y' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Z' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'X' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Y' while merging telemetry.
core        WARNING 	Failed to preserve data type for column 'Z' while merging telemetry.
